# Tradutor de piscadas para a síndrome do encarceramento com opção da escuta

A **síndrome do encarceramento** trata-se de uma rara síndrome caracterizada pela paralisação de todos os músculos voluntários, exceto do movimento ocular e o movimento de piscar.

A idéia desse sistema é monitorar os olhos do paciente que possui essa sindrome e identificar o código morse transmitido utilizando os olhos. Após obter o vídeo, esse dado é enviado para o Google Gemini, que consegue entender a mensagem. A resposta é convertida em audio e reproduzida para as pessoas ao seu redor.

Configurações iniciais

In [1]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
!pip install gTTS
"""

from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('API_KEY'))

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Código morse. Olho fechado significa ponto, olho aberto significa traço"

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

Após análise da documentação do Google Gemini a identificação do vídeo é baseado na extração de imagens a cada segundo, ou seja, se um vídeo possui 30 segundos de duração, então serão extraidos 30 fotos para análise de imagem.
No código abaixo é feito o upload de uma imagem que possui a sequencia de olhos abertos e olhos fechados.

In [2]:
genai.upload_file("olho.jpg")
convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": [genai.upload_file("olho.jpg")]
  },
])

Nesse momento perguntamos o Google Gemini qual a mensagem da imagem. O gemini então identifica o código morse dos olhos.

In [9]:
convo.send_message("qual a mensagem descifrada")
print(convo.last.text)

A mensagem em código Morse na imagem é:

**-- ..-**

Que traduz para:

**MORSE**



Com a resposta pronta, utilizando o TTS é possível converter o texto em voz

In [10]:
from gtts import gTTS
from IPython.display import Audio
tts = gTTS(convo.last.text)
tts.save('1.wav')
sound_file = '1.wav'
Audio(sound_file, autoplay=True)